In [2]:
#Final turn in version
#Objectives: 
#1. Clean up disorganized proto code
#2. Seperate blob function into seperate designated functions
#3. Figure out why geo-map no render
#4. document issues
#5. Debug
#6. -Turn in on time- new: turn in before end of week 8

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read())



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('CS-340 Dashboard_Mark M'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        className='row',
        style={'display': 'flex'},
        children=[
                html.Button(id='submit-button-one',n_clicks=0, children= 'Water Rescue'),
                html.Button(id='submit-button-two',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
                html.Button(id='submit-button-three',n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children='reset')          
        ]


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
         page_size=100,
        style_table={'height':'300px','overflowY':'auto','overflowX':'auto'},
        style_header={
            'backgroundColor':'rgb(230,230,230)',
            'fontWeight':'bold'        
        },
        style_data={
            'whiteSpace':'normal',
            'height':'auto'
        },
        
        #tooltips that we are going to use on the table so that we know what information we are looking at
        tooltip ={i: {
        'value': i,
        'use_with': 'both'  # both refers to header & data cell
    } for i in df.columns},
        tooltip_delay=0,
        tooltip_duration = None,
        
        #sorting features that we are going to use
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable=False,
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

"""@app.callback(
    dash.dependencies.Output('dd-output-container', 'children'),
    [dash.dependencies.Input('rd', 'value')]
    )
#Prototype function
def update_graphs(value):
    df = pd.DataFrame.from_records(shelter.read({}))
    
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"Labrador Retriever Mix"},
                                                        {"breed":"Chesapeake Bay Retriever"},
                                                        {"breed":"Newfoundland"}
                                                       ],"age_upon_outcome_in_weeks":{'$gt':26,'$lt':156},
                                                    "sex_upon_outcome":"Intact Female"}))
    if value == 'Mountain or Wilderness Rescue':
    
        df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"German Shepherd"},
                                                        {"breed":"Alaskan Malamute"},
                                                        {"breed":"Old English Sheepdog"},
                                                        {"breed":"Siberian Husky"} ,
                                                        {"breed":"Rottweiler"}  
                                                       ],"age_upon_outcome_in_weeks":{'$gt':26,'$lt':156},
                                                    "sex_upon_outcome":"Intact Male"}))
    if value == 'Disaster Rescue or Individual Tracking':
        df = pd.DataFrame.from_records(shelter.read({'$or':[{"breed":"German Shepherd"},
                                                        {"breed":"Doberman Pinscher"},
                                                        {"breed":"Golden Retriever"},
                                                        {"breed":"Bloodhound"} ,
                                                        {"breed":"Rottweiler"}  
                                                       ],"age_upon_outcome_in_weeks":{'$gt':20,'$lt':300},
                                                    "sex_upon_outcome":"Intact Male"}))
        
    return [
            dash_table.DataTable(
                id='datatable-id',
                columns=[
                    {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                ],
                data=df.to_dict('records'),
                editable=True,
                style_cell={'textAlign': 'left'},
                 style_cell_conditional=[
                {
                    'if': {'column_id': s},
                    'textAlign': 'left'
                } for s in ['animal_type','color','breed','animal_id']
            ],
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(0, 128, 0)'
                }
            ],
            style_header={
                'backgroundColor': 'rgb(0, 238, 0)',
                'fontWeight': 'bold'
            },
            
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable="single",
            row_selectable="multi",
            row_deletable=True,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current= 0,
            page_size= 10,

            )
        
       ]




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
"""

#Button unresponsive: fix here? ->fixed    
@app.callback([Output('datatable-id','data')],
              [Input('submit-button-one', 'n_clicks'),Input('submit-button-two','n_clicks'), 
               Input('submit-button-three','n_clicks'),Input('submit-button-four','n_clicks'),
               Input('filter-type', 'value')])
                               
def update_dashboard(bt1,bt2,bt3,bt4):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
                        
    if (int(bt1) >= 1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
                   {'breed':'Newfoundland'}]}, 
            {'sex_upon_outcome':'Intact Female'}, 
            {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        bt2, bt3, bt4 = 0

    elif (int(bt2)>= 1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'German Shepherd'}, 
                     {'breed':'Alaskan Malamute'},
                   {'breed':'Old English Sheepdog'},
                     {'breed':'Siberian Husky'},
                     {'breed':'Rottweiler'}]}, 
                    {'sex_upon_outcome':'Intact Male'}, 
                    {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        bt1, bt3 ,bt4 = 0

    elif (int(bt3)>=1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
                   {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}))
        bt1, bt2, bt4 = 0

    elif(int(bt4)>=1):
        df = pd.Dataframe.from_records(shelter.read())
        bt1, bt2, bt3 = 0

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')


    return data




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(df, values=values, names=names, title='Percentage of breeds available')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
                               
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[rows]
    
    return [ dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], zoom=15, children=[
        dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and pop up
        dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
        dl.Tooltip(dff['breed']),        
        dl.Popup([
        html.H1("Animal Name"),
        html.P(dff.loc[0,'name'])
            ])
                ])
                   ])]


app
#notes:
"""
Task: Divide up the primary update module from proto document into seperate definitions.

Possibly incomplete, fixed base issues, but started getting erros such as:
    32 # class read method must support return of cursor object
---> 33 df = pd.DataFrame.from_records(shelter.read())
 for some reason. Research and attempts have found no fixes aside from t being a bug, possibly from running an older version of a browser.
 
 *possibilty that buttons don't work fully? pacthed in possible fix, but unable to test due to above ?bug?
 
 *Logo.png not working? where does it get the png from? _> item not in jupyter folder
"""

SyntaxError: invalid syntax (<ipython-input-2-e775dc5f403f>, line 49)